In [1]:

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import scipy.stats as st
from pprint import pprint
import requests
import gmaps
import os

# Import API key
from api_keys import weather_api_key
from api_keys import g_key
#from api_keys import atlas_key
#from api_keys import nyt_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

#range of latitude and longitude for California
lat_range = (32, 42)
lng_range = (-114, -124)

In [2]:
# lat_lngs = [] #this is where we pull the coordinates
# cities = [] #we are using the coordinates to loop to the list of cities

# lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
# lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
# lat_lngs = zip(lats, lngs)

# for lat_lng in lat_lngs:
#     city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
#     # If the city is unique, then add it to a our cities list
#     if city not in cities:
#         cities.append(city)

# # Print the city count to confirm sufficient count
# len(cities)

In [3]:
# url = 'http://api.openweathermap.org/data/2.5/weather?' 

# #build query URL
# #"appid" is the application id, this is our API after the url 
# #"&q=" is the query - extensions of the URL help determine specific content or action based on the data being delivered
# #in this case our query is the city
# query_url = url + "appid=" + weather_api_key + "&q=" + city
# query_url = f"{url}appid={weather_api_key}&units=imperial&q="
# print(query_url)

In [4]:
# city_name = []
# latitude = []
# longitude = []


# x = 0

# print('Beginning Data Retrieval')
# print("-"*25)
# for i in cities:
#     try:
#         weather_response = requests.get(query_url + i).json() #each city that is in my cities list
#         x = x + 1
#         print(f'Processing Record {x} | {i}')
#         city_name.append(weather_response["name"])
#         latitude.append(weather_response["coord"]["lat"])
#         longitude.append(weather_response["coord"]["lon"])
#     except KeyError:
#         print("City not found")

In [5]:
# ca_cities_df = pd.DataFrame({
#     "City": city_name,
#     "Lat": latitude,
#     "Lng": longitude
# })
# ca_cities_df

In [6]:
#export data to csv
# ca_cities_df.to_csv('ca_cities_output.csv')


In [7]:
# gmaps.configure(api_key=g_key) 

# # Store latitude and longitude in locations, and humidity as humidity
# locations = ca_cities_df[["Lat", "Lng"]]

In [8]:
# wildfires_df = ca_cities_df.loc[:,["City", "Lat", "Lng"]]

# wildfires_df["Wildfires"] = ""

# # Display the result
# wildfires_df.head()

In [9]:
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# params = {
#           #"type" : "wildfire",
#           "keyword" : "california fire",
#           "radius" : 500,
#           "key" : g_key}

In [10]:
# for index, row in wildfires_df.iterrows():
#     # get city name, lat, lnt from df
#     lat = row["Lat"]
#     lng = row["Lng"]
#     city_name = row["City"]
    
#     params["location"] = f"{lat},{lng}"
#     print(f"Retrieving Results for Index {index}: {city_name}.")
#     response = requests.get(base_url, params=params).json()
#     results = response['results']
    
#     try:
#         print(f"Wildfires in {city_name} is {results[0]['name']}.")
#         wildfires_df.loc[index, "Wildfires"] = results[0]['name']

#     # if there is no hotel available, show missing field
#     except (KeyError, IndexError):
#         print("No wildfire found")
        
#     print("------------")

In [11]:
past_30_years = pd.read_csv('past_30_years.csv')

past_30_years_df = pd.DataFrame(past_30_years)
past_30_years_df

,Unnamed: 0,fire_name,cause,month,year,county,acres,structures,deaths
0,0,Mendocino Complex,Under Investigation,July,2018,"Colusa County, Lake County, Mendocino County &...",459123,280,1
1,1,Thomas,Powerlines,December,2017,Ventura & Santa Barbara,281893,1063,2
2,2,Cedar,Human Related,October,2003,San Diego,273246,2820,15
3,3,Rush,Lightning,August,2012,Lassen,271911,0,0
4,4,Rim,Human Related,August,2013,Tuolumne,257314,112,0
5,5,Zaca,Human Related,July,2007,Santa Barbara,240207,1,0
6,6,Carr,Human Related,July,2018,"Shasta County,Trinity County",229651,1614,8
7,8,Witch,Powerlines,October,2007,San Diego,197990,1650,2
8,9,Klamath Theater Complex,Lightning,June,2008,Siskiyou,192038,0,2
9,12,Basin Complex,Lightning,June,2008,Monterey,162818,58,0


In [12]:
del past_30_years_df['Unnamed: 0']
past_30_years_df

,fire_name,cause,month,year,county,acres,structures,deaths
0,Mendocino Complex,Under Investigation,July,2018,"Colusa County, Lake County, Mendocino County &...",459123,280,1
1,Thomas,Powerlines,December,2017,Ventura & Santa Barbara,281893,1063,2
2,Cedar,Human Related,October,2003,San Diego,273246,2820,15
3,Rush,Lightning,August,2012,Lassen,271911,0,0
4,Rim,Human Related,August,2013,Tuolumne,257314,112,0
5,Zaca,Human Related,July,2007,Santa Barbara,240207,1,0
6,Carr,Human Related,July,2018,"Shasta County,Trinity County",229651,1614,8
7,Witch,Powerlines,October,2007,San Diego,197990,1650,2
8,Klamath Theater Complex,Lightning,June,2008,Siskiyou,192038,0,2
9,Basin Complex,Lightning,June,2008,Monterey,162818,58,0
